In [1]:
import numpy as np
import glob
import os
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices(device_type='GPU') #本地需要这样操作
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu ,True)

In [2]:
x_train_path=glob.glob('./dataset/dc_2000/train/*/*.jpg')
x_test_path=glob.glob('./dataset/dc_2000/test/*/*.jpg')

In [3]:
#cat 0
#dog 1
y_train=[int(p.split('\\')[1]=='dog') for p in x_train_path]
y_train=tf.reshape(y_train,[-1,1])  #label : [1, 2, 3]==>[ [1], [2], [3]]
y_test=[int(p.split('\\')[1]=='dog') for p in x_test_path]
y_test=tf.reshape(y_test,[-1,1])  #label : [1, 2, 3]==>[ [1], [2], [3]]

In [4]:
def load_preprosess_image(img_path):
    img_raw=tf.io.read_file(img_path)   
    img_tensor=tf.image.decode_jpeg(img_raw,channels=3)
    img_tensor=tf.image.resize(img_tensor,[256,256])
    img_tensor=tf.cast(img_tensor,tf.float32)
    img_tensor=img_tensor/255
    return img_tensor

In [5]:
#自动并行运行
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [6]:
train_path_dataset=tf.data.Dataset.from_tensor_slices(x_train_path)
x_train_dataset=train_path_dataset.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)
y_train_dataset=tf.data.Dataset.from_tensor_slices(y_train)

In [ ]:
dataset_train=tf.data.Dataset.zip((x_train_dataset,y_train_dataset))

In [7]:
test_path_dataset=tf.data.Dataset.from_tensor_slices(x_test_path)
x_test_dataset=test_path_dataset.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)
y_test_dataset=tf.data.Dataset.from_tensor_slices(y_test)
dataset_test=tf.data.Dataset.zip((x_test_dataset,y_test_dataset))

In [8]:
BATCH_SIZE=4
train_count=len(x_train_path)
dataset_train=dataset_train.shuffle(train_count).repeat().batch(BATCH_SIZE).prefetch(AUTOTUNE)
#prefetch是预加载
test_count=len(x_test_path)
dataset_test=dataset_test.repeat().batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [9]:
#keras内置经典网络实现
conv_base=tf.keras.applications.VGG16(include_top=False,weights='imagenet',) #include_top表示是否加载顶层分类器，False表示只引入底层卷积基
conv_base.trainable=False

In [10]:
conv_base.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [11]:
model=tf.keras.Sequential()
model.add(conv_base)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 263,169
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['acc',]
)



In [ ]:
his=model.fit(
    dataset_train,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=15,
    validation_data=dataset_test,
    validation_steps=test_count//BATCH_SIZE
)


#微调微调微调微调微调

In [15]:
conv_base.trainable=True

In [16]:
len(conv_base.layers)

19

In [17]:
for layer in conv_base.layers[:-3]:
    layer.trainable=False

In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['acc',]
)

In [19]:
initial_epoch=15
new_epoch=10
total_epoch=initial_epoch+new_epoch

In [ ]:
model.fit(
    dataset_train,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=total_epoch,
    initial_epoch=initial_epoch,
    validation_data=dataset_test,
    validation_steps=test_count//BATCH_SIZE
)
